In [ ]:
%pip install selenium webdriver-manager pandas

In [176]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import random
import time

In [177]:
# Initialize wevdriver
def init_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    
    return webdriver.Chrome(service= Service(ChromeDriverManager().install()), options=options)

In [178]:
def count_topics(driver, course):
    course.click()

    # Спроба натиснути "Показати більше", якщо кнопка є
    try:
        show_more_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "CourseModulesList_showMore")]'))
        )
        driver.execute_script("arguments[0].click();", show_more_button)
        time.sleep(1)
    except:
        pass
        
    # Підрахунок елементів
    list_count = len(driver.find_elements(By.XPATH, '//div[contains(@class, "CourseProgram_modules")]/ul/li'))
    
    driver.back()
    
    return list_count

In [179]:
def parse_courses(driver, url):
    driver.get(url)
    actions = ActionChains(driver)

    wait = WebDriverWait(driver, 10)
    courses = []

    wait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "ProfessionCard_cardWrapper")]')))
    elements = driver.find_elements(By.XPATH, '//a[contains(@class, "ProfessionCard_cardWrapper")]')

    for i in range(len(elements)):  # для уникнення StaleElementReferenceException
        elements = driver.find_elements(By.XPATH, '//a[contains(@class, "ProfessionCard_cardWrapper")]')  # Оновлюємо список курсів
        course = elements[i]  # Отримуємо актуальний елемент       

        try:
            driver.execute_script("arguments[0].scrollIntoView();", course)
            time.sleep(0.5)
            actions.move_to_element(course).perform()
            time.sleep(1)

            name = next((el.text.strip() for el in course.find_elements(By.XPATH, './/h3[contains(@class, "ProfessionCard_title")]')), "N/A")
            description = next((el.text.strip() for el in course.find_elements(By.XPATH, './/p[contains(@class, "ProfessionCard_description")]')), "Опис відсутній")
            
            moduls = count_topics(driver, course)

            courses.append({'Name': name, 
                            'Description': description, 
                            'Number of Moduls': moduls})
        except:
            print("Parsing mistake")
            continue

    return courses

In [180]:
url = ('https://mate.academy/')
driver = init_driver()
courses_df = parse_courses2(driver, url)


In [181]:
df = pd.DataFrame(courses_df)
df.head(10)

,Name,Description,Number of Moduls
0,UI/UX designer,,13
1,Data analyst,"Аналізуй дані, які допоможуть бізнесу приймати...",16
2,QA engineer,"Перевіряй програмне забезпечення на помилки, щ...",23
3,Python developer,Пиши й підтримуй код на Python для різного про...,21
4,Digital marketer,"Створюй маркетингові кампанії, щоб допомагати ...",11
5,Front-end developer,Розробляй видимі частини вебсторінок і мобільн...,23
6,Full-stack developer,Розробляй продукт самостійно — від створення в...,26
7,DevOps engineer,"Налагоджуй інструменти і процеси, щоб програмн...",21
8,Java developer,Розробляй надійне програмне забезпечення на Ja...,19
9,Recruiter,Шукай і знаходь талановитих кандидатів для кон...,9


In [182]:
df.to_csv("courses.csv", index=False, encoding="utf-8")

In [183]:
driver.quit()